In [14]:
import sys
sys.path.append('/home/ec2-user/SageMaker')
from toniutils import dirtoni as dirt
from toniutils import printoni

<h2>Load Data</h2>

In [15]:
from script import load

In [16]:
path = 'engineered_data/experiment-1/'
X_train, X_val, Y_train, Y_val = [load(path + i + '.pkl') for i in ('X_train', 'X_val', 'Y_train', 'Y_val')]

<h2>Train Model</h2>

In [17]:
import tensorflow as tf
import numpy as np

In [18]:
from tensorflow import float32 as fl32
from tensorflow import random_normal as rnorm
from tensorflow import matmul, add
from tensorflow.nn import relu,dropout, tanh
from tensorflow.initializers import he_uniform, glorot_uniform
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15865834619247347490, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 10575286321753043484
 physical_device_desc: "device: XLA_CPU device"]

In [19]:
batch_size = 32

In [20]:
inputs = tf.placeholder(shape = [batch_size, X_train.shape[1]], dtype = fl32, name = 'inputs')
labels = tf.placeholder(shape = [batch_size, Y_train.shape[1]], dtype = fl32, name = 'labels')

In [8]:
printoni(dirt(tf.initializers), 4)

constant         identity        random_normal        variance_scaling 
global_variables lecun_normal    random_uniform       zeros            
glorot_normal    lecun_uniform   tables_initializer                    
glorot_uniform   local_variables truncated_normal                      
he_normal        ones            uniform_unit_scaling                  
he_uniform       orthogonal      variables                             


In [21]:
with tf.variable_scope('l1', reuse = tf.AUTO_REUSE):
    w1 = tf.get_variable('w1', [inputs.shape[1].value, 110], initializer = he_uniform())
    b1 = tf.get_variable('b1', [110,], initializer = tf.initializers.zeros)
    layer1 = dropout( relu(add(matmul(inputs, w1), b1), name = 'relu'), rate = 0.1 )

with tf.variable_scope('l2', reuse = tf.AUTO_REUSE):
    w2 = tf.get_variable('w2', [layer1.shape[1].value, 220], initializer = he_uniform())
    b2 = tf.get_variable('b2', [220,], initializer = tf.initializers.zeros)
    layer2 = dropout( relu(add(matmul(layer1, w2), b2), name = 'relu'), rate = 0.1)

with tf.variable_scope('l3', reuse = tf.AUTO_REUSE):
    w3 = tf.get_variable('w3', [layer2.shape[1].value, 220], initializer = he_uniform())
    b3 = tf.get_variable('b3', [220,], initializer = tf.initializers.zeros)
    layer3 = dropout( relu(add(matmul(layer2, w3), b3), name = 'relu'), rate = 0.1)
    
'''with tf.variable_scope('l4', reuse = tf.AUTO_REUSE):
    w4 = tf.get_variable('w4', [layer3.shape[1].value, 220], initializer = he_uniform())
    b4 = tf.get_variable('b4', [220,], initializer = tf.initializers.zeros)
    layer4 = dropout( relu(add(matmul(layer3, w4), b4), name = 'relu'), rate = 0.1)'''

with tf.variable_scope('output_layer', reuse = tf.AUTO_REUSE):
    w_out = tf.get_variable('w_out', [layer3.shape[1].value, 80], initializer = glorot_uniform())
    b_out = tf.get_variable('b_out', [80,], initializer = tf.initializers.zeros)
    layer_out = add(matmul(layer3, w_out), b_out)

In [22]:
with tf.variable_scope('loss', reuse = tf.AUTO_REUSE):
    loss = tf.reduce_mean(tf.squared_difference(layer_out, labels))
    loss_minimize = tf.train.AdamOptimizer().minimize(loss)

In [23]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
saver = tf.train.Saver()

In [24]:
for epoch in range(21):
    for i in range(X_train.shape[0] // batch_size):
        sess.run([loss, loss_minimize], 
                 feed_dict = {inputs: X_train[i * batch_size:(i + 1)*batch_size, :],
                              labels: Y_train[i * batch_size:(i + 1)*batch_size, :]})

    test_loss = []   
    for i in range(X_val.shape[0] // batch_size):
        new_loss,_ = sess.run([loss, loss_minimize], 
                 feed_dict = {inputs: X_val[i * batch_size:(i + 1)*batch_size, :],
                              labels: Y_val[i * batch_size:(i + 1)*batch_size, :]})
        test_loss.append(new_loss)
#     if epoch % 10 == 0:
#         # Append the step number to the checkpoint name:
#         saver.save(sess, save_path = '/home/ec2-user/SageMaker/tf_models/my-model', global_step = epoch)
    
    mean_test_loss = np.array(test_loss).mean()
    print('Epoch: ' + str(epoch) + '  -  val_loss: ' + str(mean_test_loss))
    
saver.save(sess, save_path = '/home/ec2-user/SageMaker/tf_models/my-model', global_step = 21)    

Epoch: 0  -  val_loss: 0.021928784
Epoch: 1  -  val_loss: 0.018348886
Epoch: 2  -  val_loss: 0.01656563
Epoch: 3  -  val_loss: 0.015702656
Epoch: 4  -  val_loss: 0.015113836
Epoch: 5  -  val_loss: 0.014657549
Epoch: 6  -  val_loss: 0.014428096
Epoch: 7  -  val_loss: 0.0141695645
Epoch: 8  -  val_loss: 0.014022197
Epoch: 9  -  val_loss: 0.013792179
Epoch: 10  -  val_loss: 0.013630781
Epoch: 11  -  val_loss: 0.013542541
Epoch: 12  -  val_loss: 0.01350938
Epoch: 13  -  val_loss: 0.013457846
Epoch: 14  -  val_loss: 0.013375229
Epoch: 15  -  val_loss: 0.01334921
Epoch: 16  -  val_loss: 0.013320278
Epoch: 17  -  val_loss: 0.013264502
Epoch: 18  -  val_loss: 0.013229239
Epoch: 19  -  val_loss: 0.013252493
Epoch: 20  -  val_loss: 0.013147795


'/home/ec2-user/SageMaker/tf_models/my-model-21'

In [13]:
sess.close()

In [109]:
mean_test_loss

0.013327003

In [115]:
mean_test_loss

0.0135560045

In [124]:
mean_test_loss

0.012763917

In [33]:
mean_test_loss # 20 epochs

0.0126214735

In [44]:
mean_test_loss

0.012417628

In [49]:
mean_test_loss

0.012488576

<h2>Save Model</h2>

In [119]:
tf.io.write_graph(sess.graph, '/tmp/my-model', 'train.pbtxt')

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4205122772179942051, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 8769741372308327298
 physical_device_desc: "device: XLA_CPU device"]